In [1]:
import tensorflow as tf
import pickle
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
tf.reset_default_graph()

In [3]:
dataset = pickle.load(open("/home/divyanshu/Documents/samosa/autoencoder/new_rotated_img.p","rb"))

In [5]:
features = []
labels = []
for i in range(len(dataset[0])):
    if(np.shape(dataset[0][i])== (256,256,3)):
        features.append(dataset[0][i])
        labels.append(dataset[1][i])
for i in range(len(features)):
    features[i] = np.expand_dims(features[i], axis=0)
print(len(features))
print(len(labels))

459
459


In [253]:
def conv(input, weights, strides, bias, name):
    inp = weights[0]*weights[1]*weights[2]
    weight_init = tf.random_normal_initializer(stddev = (2.0/inp)**0.5)
    W = tf.get_variable("W", weights, initializer = weight_init)
    bias_init = tf.constant_initializer(value = 0)
    b = tf.get_variable("b", bias, initializer = bias_init)
    con = tf.nn.conv2d(input, filter = W, strides = strides, padding = 'SAME')    
    con = tf.nn.bias_add(con, b)
    return tf.nn.relu(con, name = name)

In [254]:
def max_pool_layer(input_layer, filter_shape, strides, name):
    ksize = [1, filter_shape[0], filter_shape[1], 1]
    out_layer = tf.nn.max_pool(input_layer, ksize, strides, padding = 'SAME', name = name)
    return out_layer

In [255]:
x = tf.placeholder("float", shape = [1,256,256,3], name = "Input")

In [256]:
with tf.variable_scope("Encoder"):
    with tf.variable_scope("CV1"):
        cl1 = conv(x, (3,3,3,32), [1,1,1,1],[32], name = "CL1")
        pool1 = max_pool_layer(cl1 ,[2,2], [1,2,2,1], name = "POOL1")
    with tf.variable_scope("CV2"):
        cl2 = conv(pool1, (3,3,32,64), [1,2,2,1],[64], name = "CL2")
        pool2 = max_pool_layer(cl2 ,[2,2], [1,2,2,1], name = "POOL2")
    with tf.variable_scope("CV3"):
        cl3 = conv(pool2, (3,3,64,128), [1,1,1,1],[128], name = "CL3")
with tf.variable_scope("Decoder"):
    with tf.variable_scope("CV4"):
        cl4 = conv(cl3, [3,3,128,64], [1,1,1,1],[64], name = "CL4")
        shape4 = np.shape(cl4)
        upsmpl1 = tf.image.resize_nearest_neighbor(cl4, (2*shape4[1],2*shape4[2]), name = "UPSMPL1")
    with tf.variable_scope("CV5"):
        cl5 = conv(upsmpl1, [3,3,64,32], [1,2,2,1],[32], name = "CL5")
        shape5 = np.shape(cl5)
        upsmpl2 = tf.image.resize_nearest_neighbor(cl5, (8*shape5[1],8*shape5[2]), name = "UPSMPL2")
    with tf.variable_scope("Decoded_Layer"):
        decoded = conv(upsmpl2, [3,3,32,3], [1,1,1,1],[3], name = "CL6")  

In [257]:
l2 = tf.sqrt(tf.reduce_sum(((tf.subtract(decoded,x))**2),1), name = "LTwo")
train_loss = tf.reduce_mean(l2, name = "Loss")
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
train_op = optimizer.minimize(train_loss)
init = tf.global_variables_initializer()
directory = "/home/divyanshu/Documents/samosa"

In [259]:
with tf.Session() as sess:
    sess.run(init)
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter("/tmp/log/...", sess.graph)
    for epoch in range(100):
        print("Epoch: " + str(epoch))
        for i in range(len(features)):
            train_op_, train_loss_ = sess.run([train_op, train_loss], feed_dict = {x:features[i]})
            print(train_loss_)
    writer.close()
    saver.save(sess, directory)

(1, 256, 3)


In [224]:
#The freeze model script to generate .pb file

import os, argparse
# The original freeze_graph function
# from tensorflow.python.tools.freeze_graph import freeze_graph 

directory = "/home/divyanshu/Documents/"

def freeze_graph(model_dir, output_node_names):
    """Extract the sub graph defined by the output nodes and convert 
    all its variables into constant 
    Args:
        model_dir: the root folder containing the checkpoint state file
        output_node_names: a string, containing all the output node's names, 
                            comma separated
    """
    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            "directory: %s" % model_dir)

    if not output_node_names:
        print("You need to supply the name of a node to --output_node_names.")
        return -1

    # We retrieve our checkpoint fullpath
    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path
    
    # We precise the file fullname of our freezed graph
    absolute_model_dir = "/".join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + "/frozen_model.pb"

    # We clear devices to allow TensorFlow to control on which device it will load operations
    clear_devices = True

    # We start a session using a temporary fresh Graph
    with tf.Session(graph=tf.Graph()) as sess:
        # We import the meta graph in the current default Graph
        saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)

        # We restore the weights
        saver.restore(sess, input_checkpoint)

        # We use a built-in TF helper to export variables to constants
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess, # The session is used to retrieve the weights
            tf.get_default_graph().as_graph_def(), # The graph_def is used to retrieve the nodes 
            output_node_names.split(",") # The output node names are used to select the usefull nodes
        ) 

        # Finally we serialize and dump the output graph to the filesystem
        with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))

    return output_graph_def
freeze_graph(directory, "Loss")

INFO:tensorflow:Restoring parameters from /home/divyanshu/Documents/samosa
INFO:tensorflow:Froze 12 variables.
INFO:tensorflow:Converted 12 variables to const ops.
57 ops in the final graph.


node {
  name: "Input"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 1
        }
        dim {
          size: 256
        }
        dim {
          size: 256
        }
        dim {
          size: 3
        }
      }
    }
  }
}
node {
  name: "Encoder/CV1/W"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 3
          }
          dim {
            size: 3
          }
          dim {
            size: 3
          }
          dim {
            size: 32
          }
        }
        tensor_content: ".\376\346\275:1\340<\'\006\206>Bv\033\276\007\356y\276\241$E\2761:\367=>\273\272\276\007\365\254\276\301\017\226=\374\330\177>\'\324[>\356[\345>\357\033\336\275.<,>\313\271Z\275\326J\344>\211\030\346</\